# Connect to the cluster

In [ ]:
import socket
on_io = socket.gethostname() == 'io'
if on_io and 1:
    import hpc05, hpc05_monitor
    client = hpc05.connect.start_remote_and_connect(100, folder='~/Work/two_dim_majoranas')[0]
#     max_usage_task = hpc05_monitor.start(client, interval=1)
else:
    # No not connect to the cluster in CI for example.
    client = None

## Numerics imports

In [ ]:
from functools import partial
from copy import copy
import numpy as np
import adaptive
import sns_system, kwant, spectrum
adaptive.notebook_extension()

import pathlib
pathlib.Path('paper/figures').mkdir(parents=True, exist_ok=True) 

## Plotting imports

In [ ]:
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

class HistogramNormalize(matplotlib.colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        matplotlib.colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                       len(self.sorted_data))
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm

golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
fig_width_pt = 246.0  # Columnwidth
inches_per_pt = 1.0 / 72.27  # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]

params = {
          'backend': 'ps',
          'axes.labelsize': 13,
          'font.size': 13,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 100 if on_io else 300,
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}', r'\usepackage{xfrac}'])

## Parameter and system definitions

In [ ]:
import scipy.constants
import cmath

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle=26, g_factor_left=0, g_factor_right=0,
                 mu=10,
                 alpha_middle=20, alpha_left=0, alpha_right=0,
                 Delta_left=1, Delta_right=1,
                 B_x=0,
                 B_y=0,
                 B_z=0,
                 phase=np.pi,
                 T=0.0,
                 V=0)

default_params = dict(**constants,
              **params_raw)

default_syst_pars = dict(
    L_m=200,
    L_x=1300,
    L_sc_up=200,
    L_sc_down=200,
    z_x=1300,
    z_y=0,
    a=10,
    shape='parallel_curve',
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=True,
    k_x_in_sc=True,
    wraparound=True,
    current=False,
    ns_junction=False)

# Figure 2. Bandstructures

In [ ]:
def spectrum_wrapper(k_x, z_y, B_x, phase, syst_pars=copy(default_syst_pars),
                     params=copy(default_params), nbands=40):
    import numpy as np
    import sns_system, spectrum
    params = dict(params, k_x=k_x, B_x=B_x, phase=phase)
    syst_pars = dict(syst_pars, z_y=z_y)
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    energies = spectrum.calc_spectrum(syst_at_k, params, k=nbands)[0]
    return np.sort(energies)

def abs_min_log_loss(xs, ys):
    from adaptive.learner.learner1D import default_loss
    ys = [np.log(np.abs(y).min()) for y in ys]
    return default_loss(xs, ys)

B = 1
W = default_syst_pars['L_m']
combos = [(0, 0, 0), (W/4, 0, 0), (W/2, 0, 0), (0, B, np.pi), (W/4, B, np.pi), (W/2, B, np.pi)]

learners = [adaptive.Learner1D(partial(spectrum_wrapper, z_y=z_y, B_x=B_x, phase=phase),
                               bounds=[-np.pi, np.pi], loss_per_interval=abs_min_log_loss)
            for z_y, B_x, phase in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y', 'B_x', 'phi'], combos), strategy='npoints')

fnames = [f'data/bandstructures/spectrum_{combo}.pickle' for combo in combos]
learner.load(fnames)

runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 200, executor=client)
runner.live_info()

In [ ]:
mapping = {(z_y, B_x, phase): learner for (z_y, B_x, phase), learner in zip(combos, learners)}

def plot(ax, key, color):
    data = mapping[key].data
    xs, ys = map(np.array, zip(*sorted(data.items())))
    return ax.plot(xs, ys, c=color)

fig, axs = plt.subplots(3, sharex=True, sharey=True, figsize=(fig_width, 2*fig_height))
(ax1, ax2, ax3) = axs

line2 = plot(ax1, (0, B, np.pi), 'red')[0]
line1 = plot(ax1, (0, 0, 0), 'blue')[0]

plot(ax2, (W/4, B, np.pi), 'red')
plot(ax2, (W/4, 0, 0), 'blue')

plot(ax3, (W/2, B, np.pi), 'red')
plot(ax3, (W/2, 0, 0), 'blue')

for i, ax in enumerate(axs):
    ax.set_ylabel(r'$E/\Delta$')
    
    # ylims and yticks
    ax.set_ylim(-0.35, 0.35)
    yvals = [-0.3, 0, 0.3]
    ylabels = [f'${x}$' for x in yvals]
    ax.set_yticks(yvals)
    ax.set_yticklabels(ylabels)
    
    # xlims and xticks
    ax.set_xlim(-3, 3)
    xvals = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
    xlabels = ['$\sfrac{-\pi}{a}$', '$\sfrac{-\pi}{2a}$', r'$0$', '$\sfrac{\pi}{2a}$', r'$\sfrac{\pi}{a}$']
    ax.set_xticks(xvals)
    ax.set_xticklabels(xlabels)

    # text inside image
    label = 'abc'[i]
    ax.text(0.95, 0.5, f'$\mathrm{{({label})}}$', transform=ax.transAxes,
            verticalalignment='center', horizontalalignment='center')
    
    z_ys = ['0', 'W/4', 'W/2']
    ax.text(0.01, 0.5, f'$z_y={z_ys[i]}$', transform=ax.transAxes,
            verticalalignment='center', horizontalalignment='left')

ax1.legend((line1, line2), (r'$\phi=0$, $B=0$', r'$\phi=\pi$, $B \ne 0$'),
           loc='upper center', bbox_to_anchor=(0.5, 1.7),
           fancybox=True, shadow=False, ncol=1)

ax3.set_xlabel('$k_x$')

plt.savefig("paper/figures/bandstructures.pdf", bbox_inches="tight")
plt.show()

# Figure 3. Phase diagrams

TODO:
* add the phase boundaries
* fix the algorithm (something goes wrong when calculating the gap)

### Phase boundaries

In [ ]:
syst, site_parts, _ = sns_system.make_system(**dict(default_syst_pars,
    z_y=0, mu_from_bottom_of_spin_orbit_bands=False, z_x=10, L_x=10))

part_to_color = {'middle_interior' : 'grey',
                 'middle_barrier' : 'black',
                 'bottom_superconductor' : 'gold',
                 'top_superconductor' : 'gold',
                 'top_cut' : 'red',
                 'bottom_cut' : 'blue'}

site_colors = [part_to_color[site_parts[s]] for s in syst.sites]
kwant.plot(syst, site_color=site_colors);

In [ ]:
import operator
from adaptive import Learner1D, make_datasaver

def phase_boundary_wrapper(B_x, key, z_y, syst_pars=copy(default_syst_pars),
                           params=copy(default_params), nbands=200):
    import numpy as np
    import sns_system, spectrum
    
    params = dict(params, B_x=B_x)
    syst_pars = dict(syst_pars, z_y=z_y, mu_from_bottom_of_spin_orbit_bands=False)
#     syst_pars['z_x'] = 10 # tmp
#     syst_pars['L_x'] = 10 # tmp
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    for k in ['alpha_middle', 'alpha_left', 'alpha_right']:
        params[k] = 0
    return spectrum.find_phase_bounds(
        syst_at_k, params, k_x=0, num_bands=nbands, sigma=15)


W = default_syst_pars['L_m']
combos = [(0, 'mu'),
#           (0, 'phase'),
          (W/2, 'mu'),
#           (W/2, 'phase')
]

# Learner = make_datasaver(Learner1D, arg_picker=operator.itemgetter(0))

learners = [Learner1D(function=partial(phase_boundary_wrapper, key=key, z_y=z_y),
                      bounds=(0, 2)) for z_y, key in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y', 'key'], combos), strategy='npoints')

fnames = [f'data/phase_boundary/phase_boundary_{combo}.pickle' for combo in combos]
learner.load(fnames)

runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 10000, executor=client)
runner.live_info()

In [ ]:
runner.task.print_stack()

In [ ]:
def plot(l):
    import holoviews as hv
    if l.data:
        xs, ys = map(np.array, zip(*sorted(l.data.items())))
        plots = [hv.Scatter((xs, y)) for y in ys.T]
    else:
        plots = [hv.Scatter([])]
    return hv.Overlay(plots).redim(x='B_x', y='mu')

learner.plot(plotter=plot).select(mu=(10, 20))

### Energy gaps

In [ ]:
import operator
from adaptive import Learner2D, make_datasaver

def gap_wrapper(xy, keys, z_y, syst_pars=copy(default_syst_pars), params=copy(default_params), nbands=10):
    import numpy as np
    import sns_system, spectrum
    from scipy.optimize import brute
    params = dict(params, **dict(zip(keys, xy)))
    syst_pars = dict(syst_pars, z_y=z_y)

    syst_at_k, _, _ = sns_system.make_system(**syst_pars)

    def energies(k_x):
        params['k_x'] = float(k_x)
        return np.abs(spectrum.calc_spectrum(syst_at_k,
            params, k=nbands)[0]).min()

    return brute(energies, ranges=((0, np.pi),), Ns=201, full_output=True)

W = default_syst_pars['L_m']
combos = [(0, 'mu', (10, 30)), (0, 'phase', (0, 2*np.pi)),
          (W/2, 'mu', (10, 30)), (W/2, 'phase', (0, 2*np.pi))]

Learner = make_datasaver(Learner2D, arg_picker=operator.itemgetter(1))

learners = [Learner(function=partial(gap_wrapper, keys=[key, 'B_x'], z_y=z_y),
                               bounds=[xbounds, (0, 2)])
            for z_y, key, xbounds in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y', 'key'], combos), strategy='npoints')

fnames = [f'data/phase_diagrams/phase_diagram_{combo}.pickle' for combo in combos]
learner.load(fnames)

runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 10000, executor=client)
runner.live_info()

In [ ]:
runner.task.print_stack()

In [ ]:
runner.start_periodic_saving(dict(fname=fnames), 1800)

In [ ]:
list(l.npoints for l in learner.learners)

In [ ]:
(runner.elapsed_time() / sum(l.npoints for l in learner.learners) * 10000 * 4 - runner.elapsed_time()) / 3600

### Combined plot

In [ ]:
mapping = {(z_y, key): learner for (z_y, key, xbounds), learner in zip(combos, learners)}

E_max = max(max(l.data.values()) for l in learners)

def plot(ax, z_y, key):
    learner = mapping[(z_y, key)]
    im = learner.plot().Image.I
    l, b, r, t = im.bounds.lbrt()

    return ax.imshow(im.data,
                     extent=(l, r, b, t),
                     aspect='auto',
                     cmap='viridis')

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(fig_width, 2.2*fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.8, hspace=0.22, wspace=0.12)
(ax1, ax2), (ax3, ax4) = axs

ims = [plot(ax, z_y, key) for ax, (z_y, key, xbounds) in zip(axs.reshape(-1), combos)]
    

for i, ax in enumerate(axs.reshape(-1)):
    if i in [0, 2]:
        ax.set_ylabel(r'$B_x$ (T)')

    # ylims and yticks
    yvals = [0, 1, 2]
    ylabels = [f'${x}$' for x in yvals]
    ax.set_yticks(yvals)
    ax.set_yticklabels(ylabels)

    # xlims and xticks
    if i in [1, 3]:
        xvals = [0, np.pi, 2*np.pi]
        xlabels = [r'$0$', '$\pi$', r'$2\pi$']
        ax.set_xticks(xvals)
        ax.set_xticklabels(xlabels)

    # text inside image
    label = 'abcd'[i]
    ax.text(0.99, 0.98, f'$\mathrm{{({label})}}$', transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            color='white')

    z_y = ['0', '0', 'W/2', 'W/2'][i]
    ax.text(0.99, 0.01, f'$z_y={z_y}$', transform=ax.transAxes,
            verticalalignment='bottom', horizontalalignment='right',
            color='white')

ax3.set_xlabel('$\mu$ (meV)')
ax4.set_xlabel('$\phi$ (rad)')

im = learners[1].plot().Image.I
norm = HistogramNormalize(im.data, vmin=0, vmax=E_max, mixing_degree=0.6)
for im in ims:
    im.set_norm(norm)

cax = fig.add_axes([0.83, 0.2, 0.04, 0.6])  # [l, b, w, h]
cb = fig.colorbar(ims[3], cax=cax,
                  label=r'$E_{\mathrm{gap}}/ \Delta$')
# cbar_ticks = [0, 5, 10, 15, 20, 25]
# cb.set_ticks(cbar_ticks)

plt.savefig("paper/figures/phasediagrams.pdf", bbox_inches="tight")
plt.show()

# Figure 4. Wavefunctions

TODO: add the Laeven shape and tune the parameters

In [ ]:
def num_to_latex_exp(x, only_exp=True):
    if not only_exp:
        num, exponent = f'{x:.0e}'.split('e')
        return rf'{num} \cdot 10^{{{int(exponent)}}}'
    else:
        exponent = np.floor(np.log10(np.abs(x))).astype(int)
        return f'10^{{{int(exponent)}}}'

def gist_heat_r_transparent():
    import matplotlib.cm
    import matplotlib.colors as mcolors
    colors = matplotlib.cm.gist_heat_r(np.linspace(0, 1, 128))
    colors[:, 3] = np.linspace(0, 1, 128).tolist()
    gist_heat_r_transparent = mcolors.LinearSegmentedColormap.from_list('gist_heat_r_transparent', colors)
    return gist_heat_r_transparent

def deltas(syst):
    return [np.abs(syst.hamiltonian(i, i, params=params)[1, 0])
            for i, site in enumerate(syst.sites)]

part_to_color =  {'middle_interior' : 'grey',
                  'middle_barrier' : 'black',
                  'bottom_superconductor' : 'gold',
                  'top_superconductor' : 'gold',
                  'top_cut' : 'red',
                  'bottom_cut' : 'blue'}

fig, axs = plt.subplots(4, 1, sharex=True, sharey=True, figsize=(fig_width, 2*fig_height))
(ax1, ax2, ax3, ax4) = axs

shapes = ['parallel_curves',
          'sawtooth',
          'parallel_curves',
          'parallel_curves']  # XXX: last one should be Laeven shape

for i, (ax, shape, z_y) in enumerate(zip(axs, shapes, [0, 100, 100, 180])):
    L_x = 3 * default_syst_pars['z_x']
    syst_pars = dict(default_syst_pars, wraparound=False, z_y=z_y, L_x=L_x, shape=shape)
    syst, site_parts, _ = sns_system.make_system(**syst_pars)
    params = dict(default_params, B_x=1)
    ham = syst.hamiltonian_submatrix(sparse=True, params=params)
    energies, wfs = spectrum.sparse_diag(ham, 4, 0)
    wfs = wfs[:, energies > 0]
    energies = energies[energies > 0]
    inds_min = np.argsort(energies)
    rho = kwant.operator.Density(syst)
    
    # text inside image
    label = 'abcd'[i]
    ax.text(0.995, 0.95, f'$\mathrm{{({label})}}$', transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            color='black')
    
    E_M = num_to_latex_exp(np.abs(energies[inds_min[0]]), only_exp=False)
    E_gap = num_to_latex_exp(np.abs(energies[inds_min[1]]), only_exp=False)

    ax.text(0.01, 0.945, rf'$E_M/\Delta={E_M}$',
            transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='left',
            color='black')
    
    ax.text(0.01, 0.02, rf'$E_{{\textrm{{gap}}}} / \Delta = {E_gap}$',
            transform=ax.transAxes,
            verticalalignment='bottom', horizontalalignment='left',
            color='black')
    
    ax.set_ylabel('$y$ (nm)')
    
    # Plot the lattice
#     site_colors = [part_to_color[site_parts[s]] for s in syst.sites]
#     site_colors = delta(syst.sites)
#     kwant.plot(syst, site_color=site_colors, ax=ax, site_lw=0)

    # Plot where the normal region is
    Deltas = deltas(syst)
    Deltas = [1 if x == 0 else np.nan for x in Deltas]
    gray_r = matplotlib.cm.gray_r
    gray_r.set_bad('#e0e0e0')
    kwant.plotter.map(syst, Deltas, ax=ax, show=False, cmap=gray_r)
    
    # Plot the wf
    kwant.plotter.map(syst, rho(wfs[:, inds_min[0]]), ax=ax, show=False, cmap=gist_heat_r_transparent())

ax4.set_xlabel('$x$ (nm)')
    
plt.savefig("paper/figures/wavefunctions.pdf", bbox_inches="tight")
plt.show()